In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, save_model, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

In [2]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [3]:
# Freeze the base model
for layer in base_model.layers:
    layer.trainable = False

In [4]:
# Add custom layers on top of the base model
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(6, activation='softmax')(x)  # Adjust the number of units based on the number of classes

In [5]:
model = Model(inputs=base_model.input, outputs=x)

In [6]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
for layer in base_model.layers[-3:]:
    layer.trainable = True

In [8]:
# Compile the model with a low learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
# Create the Sequential model
model = Sequential([
    Input(shape=(224, 224, 3)),  # Define the input shape with the Input layer
    
    Conv2D(32, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Flatten(),

    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.5),

    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.5),

    Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.5),

    Dense(6, activation='softmax')
])

In [10]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 222, 222, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 109, 109, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 52, 52, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 32)             │             

 Total params: 11,181,286 (42.65 MB)

 Trainable params: 11,180,390 (42.65 MB)

 Non-trainable params: 896 (3.50 KB)

In [12]:
# Data Augmentation and Generators
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, 
                                   horizontal_flip=True, fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('../DL dataset/train', target_size=(224, 224), 
                                                    batch_size=64, class_mode='categorical')

validation_generator = test_datagen.flow_from_directory('../DL dataset/val', target_size=(224, 224), 
                                                        batch_size=64, class_mode='categorical')

Found 30909 images belonging to 6 classes.
Found 3923 images belonging to 6 classes.


In [13]:
data_dir = '../DL dataset/train'

print("Class indices:", train_generator.class_indices)
for cls in train_generator.class_indices:
    cls_dir = os.path.join(data_dir, cls)
    num_images = len(os.listdir(cls_dir))
    print(f"Class {cls} has {num_images} images")

Class indices: {'1. Enfeksiyonel': 0, '2. Ekzama': 1, '3. Akne': 2, '4. Pigment': 3, '5. Benign': 4, '6. Malign': 5}
Class 1. Enfeksiyonel has 6000 images
Class 2. Ekzama has 4070 images
Class 3. Akne has 2149 images
Class 4. Pigment has 1020 images
Class 5. Benign has 10888 images
Class 6. Malign has 6783 images


In [14]:
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)

class_weights_dict = dict(enumerate(class_weights))
print("Class weights:", class_weights_dict)

Class weights: {0: np.float64(0.8585833333333334), 1: np.float64(1.2657248157248158), 2: np.float64(2.3982774674115457), 3: np.float64(5.050490196078432), 4: np.float64(0.47313556208670093), 5: np.float64(0.7594722099366062)}


In [15]:
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True,verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6,verbose=1)

In [16]:
! pip install Pillow

You should consider upgrading via the 'C:\Users\vivek kumar\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [19]:
 # Train the Model
import PIL
history = model.fit(train_generator, epochs=10, validation_data=validation_generator, callbacks=[reduce_lr,early_stopping],class_weight = class_weights_dict)

Epoch 1/10
483/483 ━━━━━━━━━━━━━━━━━━━━ 2033s 4s/step - accuracy: 0.3537 - loss: 1.9296 - val_accuracy: 0.3564 - val_loss: 1.8077 - learning_rate: 0.0010
Epoch 2/10
483/483 ━━━━━━━━━━━━━━━━━━━━ 2089s 4s/step - accuracy: 0.3676 - loss: 1.7915 - val_accuracy: 0.4428 - val_loss: 1.6227 - learning_rate: 0.0010
Epoch 3/10
483/483 ━━━━━━━━━━━━━━━━━━━━ 2039s 4s/step - accuracy: 0.3913 - loss: 1.7895 - val_accuracy: 0.4410 - val_loss: 1.5136 - learning_rate: 0.0010
Epoch 4/10
483/483 ━━━━━━━━━━━━━━━━━━━━ 1957s 4s/step - accuracy: 0.3917 - loss: 1.7712 - val_accuracy: 0.4688 - val_loss: 1.6653 - learning_rate: 0.0010
Epoch 5/10
483/483 ━━━━━━━━━━━━━━━━━━━━ 2012s 4s/step - accuracy: 0.4192 - loss: 1.7865 - val_accuracy: 0.4912 - val_loss: 1.6014 - learning_rate: 0.0010
Epoch 6/10
483/483 ━━━━━━━━━━━━━━━━━━━━ 1949s 4s/step - accuracy: 0.4450 - loss: 1.7750 - val_accuracy: 0.2982 - val_loss: 2.2678 - learning_rate: 0.0010
Epoch 7/10
483/483 ━━━━━━━━━━━━━━━━━━━━ 2110s 4s/step - accuracy: 0.4438 - l

In [20]:
# Save the Trained Model
model.save("trained_model.keras")

In [21]:
# Save the model in TensorFlow SavedModel format
model.save('model.h5')

In [22]:
# Load the model from TensorFlow SavedModel format
model = tf.keras.models.load_model('model.h5')

In [24]:
# Evaluate the Model on Test Data
test_generator = test_datagen.flow_from_directory('../DL dataset/test', target_size=(224, 224), 
                                                  batch_size=64, class_mode='categorical', shuffle=True)

test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {test_accuracy * 100:.2f}%')

Found 3928 images belonging to 6 classes.


c:\Users\vivek kumar\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


62/62 ━━━━━━━━━━━━━━━━━━━━ 121s 2s/step - accuracy: 0.5608 - loss: 1.3792
Test accuracy: 56.42%


In [25]:
# Get the ground truth labels
test_labels = test_generator.classes
print(test_labels)

[0 0 0 ... 5 5 5]


In [26]:
# Predict the classes
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

62/62 ━━━━━━━━━━━━━━━━━━━━ 50s 796ms/step


In [28]:

# Print classification report
print(classification_report(test_labels, predicted_classes, target_names=test_generator.class_indices.keys()))

# Print confusion matrix
print(confusion_matrix(test_labels, predicted_classes))

                 precision    recall  f1-score   support

1. Enfeksiyonel       0.17      0.22      0.20       750
      2. Ekzama       0.12      0.15      0.14       510
        3. Akne       0.08      0.11      0.09       322
     4. Pigment       0.04      0.04      0.04       136
      5. Benign       0.36      0.22      0.27      1361
      6. Malign       0.19      0.21      0.20       849

       accuracy                           0.19      3928
      macro avg       0.16      0.16      0.16      3928
   weighted avg       0.22      0.19      0.20      3928

[[167 128  87  20 165 183]
 [116  79  58  22 104 131]
 [ 90  42  36  14  65  75]
 [ 32  19  17   5  31  32]
 [313 222 150  43 299 334]
 [242 150  98  24 156 179]]
